This data product is meant as a beginner project and is largley taken from http://adventuresinmachinelearning.com/python-tensorflow-tutorial/


For this predictive algorithm, we will use numpy and tensorflow.

In [11]:
import numpy as np
import tensorflow as tf
from IPython.core.debugger import set_trace

We then set our learning parameters - these can be changed to optimize our processing speeds

In [12]:
learning = 0.5
epochs = 10
batch_size = 100

We need placeholders for the trainng algorithm - the x placeholder is the input matrix while y is the output matrix that will determine which team will win.

In [13]:
x = tf.placeholder(tf.float32,[None,312])
y = tf.placeholder(tf.float32,shape=[None,1],name = "labels")

Our model will have 4 layers: the input layer, two hidden layers ,sizes 200 and 100 respectivley(will be changed later to see its effect on the prediction) and one output layer.

In [14]:
# connecting input to first layer
W1 = tf.Variable(tf.random_normal([312, 200], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([200]), name='b1')

# connecting first to second layer
W2 = tf.Variable(tf.random_normal([200, 100], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([100]), name='b2')

# connecting second to output layer
W3 = tf.Variable(tf.random_normal([100, 1], stddev=0.03), name='W3')
b3 = tf.Variable(tf.random_normal([1]), name='b3')

With these random weights, we can perform forward propagation to calculate each of the hidden layers and the final output layer. We use the relu activation function in calculating hidden layers. In the end we use the softmax function to get an prediction on the outcome of the game. 

In [15]:
#input to first layer
hidden_one = tf.add(tf.matmul(x,W1),b1)
hidden_one = tf.nn.relu(hidden_one)

#first to second layer
hidden_two = tf.add(tf.matmul(W1,W2),b2)
hidden_two = tf.nn.relu(hidden_two)

#output layer
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_two,W3),b3))

Since our cost function uses the log(x) function, we need to ensure that the log(0) scenario does not happen. This will result in an NaN error. Clipping these values will negate these errors

In [16]:
y_clipped = tf.clip_by_value(y_,1e-10,0.9999999)

We need to set the cost function and optimizer.

Our cost function is the cross-entropy function 
    \begin{align}
    J = -\frac{1}{m} \sum_{i=1}^m \sum_{j=1}^n y_j^{(i)}log(y_j\_^{(i)}) + (1 – y_j^{(i)})log(1 – y_j\_^{(i)})
    \end{align}
    
Where the ith variable is the predicted value of the current training example.

In [17]:
#define cost function
cost_func = tf.reduce_mean(tf.reduce_sum(y*tf.log(y_clipped) + (1-y) * tf.log(1-y_clipped),axis=1))

#optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning).minimize(cost_func)

We now create an accuracy testing operation. We can test how well our algorithm is performing by comparing the predicted result with the correct result in our testing set.

In [18]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

We then get our training set and labels from their respective folders. They are stored in .csv files and we can convert them into numpy arrays

In [19]:
trainingSet = np.genfromtxt("DataSets/trainingSet - Copy.csv",delimiter=',', dtype="float32")
labels = np.genfromtxt("Labels/trainingSet - Copy.csv",delimiter=',', dtype="float32")

labels = np.reshape(labels,(len(labels),1))

test_x = np.genfromtxt("DataSets/testingSet - Copy.csv", delimiter=',', dtype = "float32")
test_y = np.genfromtxt("Labels/testingSet - Copy.csv",delimiter=',',dtype = "float32")

test_y = np.reshape(test_y,(len(test_y),1))

We need to convert the labels set from a np array with one column to an array to two columns. 

We can now run our session, training in batches.

In [20]:
init_op = tf.global_variables_initializer()
prediction = y_

with tf.Session() as sess:
    sess.run(init_op)

    for epoch in range(epochs):
        epoch_loss = 0
        i=0
        while i < len(trainingSet):
            start = i
            end = i+batch_size
            batch_x = np.array(trainingSet[start:end])
            batch_y = np.array(labels[start:end])
            
            #set_trace()
            _, c = sess.run([optimizer, cost_func], feed_dict={x: batch_x,
                                                          y: batch_y})
            epoch_loss += c
            i+=batch_size

        print('Epoch', epoch+1, 'completed out of',epochs,'loss:',epoch_loss)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

    print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

InvalidArgumentError: Incompatible shapes: [100,1] vs. [312,1]
	 [[Node: gradients_1/mul_2_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients_1/mul_2_grad/Shape, gradients_1/mul_2_grad/Shape_1)]]

Caused by op 'gradients_1/mul_2_grad/BroadcastGradientArgs', defined at:
  File "c:\program files\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\program files\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\program files\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\program files\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\program files\python36\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "c:\program files\python36\lib\site-packages\tornado\ioloop